In [1]:
import pandas as pd
import numpy as np
import cv2
import os

import tensorflow as tf
from tensorflow.keras import models, layers, datasets

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

from plotting import show_distribution
from plot_val import calc_roc_curve, plot_roc_curve, plot_confusion_matrix

from apply_processing import apply, resize_images, function_hls, function_gray
from apply_processing import function_label, blur_images, adjust_contrast_images, rotated_labels
from apply_processing import filter_images, edge_images, contour_images, rotate_images

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(f"Nums GPUs available : {len(physical_devices)}")

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Nums GPUs available : 1


In [3]:
df = pd.read_csv("Dataset\\images_mushrooms.csv")

df.head()

,Image,Label
0,Dataset\Mushrooms\Agaricus\009_mrv34Sn4WiQ.jpg,Agaricus
1,Dataset\Mushrooms\Agaricus\010_Rk1D3EG856g.jpg,Agaricus
2,Dataset\Mushrooms\Agaricus\017_QHGfbEiWvq8.jpg,Agaricus
3,Dataset\Mushrooms\Agaricus\020_vZJfrjIwmqU.jpg,Agaricus
4,Dataset\Mushrooms\Agaricus\030_-i6kYqDvBQo.jpg,Agaricus


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Image   900 non-null    object
 1   Label   900 non-null    object
dtypes: object(2)
memory usage: 14.2+ KB


In [5]:
df['Label'].value_counts() / df.shape[0]

Entoloma       0.111111
Boletus        0.111111
Cortinarius    0.111111
Russula        0.111111
Hygrocybe      0.111111
Suillus        0.111111
Amanita        0.111111
Lactarius      0.111111
Agaricus       0.111111
Name: Label, dtype: float64

# Processing the images

In [6]:
# With Contour + Fixed contrast, brightness, sharpness

def processing_contour(df):
    images = df['Image']
    labels = df['Label']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 10
    
    images_hls = function_hls(images)
    images_hls = resize_images(images_hls, size)
    images_hls = filter_images(images_hls)
    images_hls = adjust_contrast_images(images_hls, contrast=contrast, brightness=brightness)
    images_hls = blur_images(images_hls)
    
    images_gray = function_gray(images)
    images_gray = resize_images(images_gray, size)
    images_gray = filter_images(images_gray)
    images_gray = adjust_contrast_images(images_gray, contrast=contrast, brightness=brightness)
    images_gray = blur_images(images_gray)
    
    edged_images = edge_images(images_gray)
    
    images_contour = contour_images(edged_images, images_hls)
    
    return list(images_contour)

In [7]:
# Random Contrast, Brightness, Sharpness

def processing_random(df):
    images = df['Image']
    labels = df['Label']
    
    size = (150, 150)
    contrast = np.random.randint(1, 5)
    brightness = np.random.randint(-100, 100)
    
    images_hls = function_hls(images)
    images_hls = resize_images(images_hls, size)
    images_hls = filter_images(images_hls)
    images_hls = adjust_contrast_images(images_hls, contrast=contrast, brightness=brightness)
    images_hls = blur_images(images_hls)
    
    return list(images_hls)

In [8]:
# Fixed contrast, brightness, sharpness

def processing_fixed(df):
    images = df['Image']
    labels = df['Label']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 10
    
    images_hls = function_hls(images)
    images_hls = resize_images(images_hls, size)
    images_hls = filter_images(images_hls)
    images_hls = adjust_contrast_images(images_hls, contrast=contrast, brightness=brightness)
    images_hls = blur_images(images_hls)
    
    return list(images_hls)

In [9]:
# Rotated and flipped images

def processing_rotated(df):
    images = df['Image']
    labels = df['Label']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 10
    
    images_hls = function_hls(images)
    images_hls = resize_images(images_hls, size)
    images_hls = filter_images(images_hls)
    images_hls = adjust_contrast_images(images_hls, contrast=contrast, brightness=brightness)
    images_hls = blur_images(images_hls)
    images_hls = rotate_images(images_hls)
    
    return list(images_hls)

In [10]:
functions = [processing_contour, processing_random, processing_fixed]
labels = [function_label, function_label, function_label]

images_aug, labels_aug = apply(functions, labels, df)

# Images info

In [11]:
print(f"Length of images no augmentation {len(images_aug)}")
print(f"Length of labels no augmentation {len(labels_aug)}")

Length of images no augmentation 2700
Length of labels no augmentation 2700


## Distribution

In [12]:
# show_distribution(images_noAug)

# Modelling

In [13]:
# le = LabelEncoder()
# le.fit(labels)

In [14]:
# labels = le.transform(labels_noAug)
# images = images_noAug / 255.0

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33, random_state=42)

In [16]:
# model = models.Sequential()
# model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(150, 150, 1)))
# model.add(layers.MaxPooling2D((3,3)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((3,3)))
# model.add(layers.Conv2D(32, (3,3), activation='relu'))

In [17]:
# model.summary()

In [18]:
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='sigmoid'))
# model.add(layers.Dense(4))

In [19]:
# model.summary()

In [20]:
# adam_callback = tf.keras.callbacks.TensorBoard(log_dir='CNN_board/adam', histogram_freq=1)
# sgd_callback = tf.keras.callbacks.TensorBoard(log_dir='CNN_board/sgd', histogram_freq=1)

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [21]:
# model_training = model.fit(X_train, y_train, epochs=6, 
#                            validation_data=(X_test, y_test), batch_size=32)

In [22]:
# model.evaluate(X_test, y_test)

## Evaluation

### ROC and AUC

In [23]:
# model_proba = model.predict(X_test)

In [24]:
# fprs, tprs, aucs = calc_roc_curve(y_test, model_proba)

In [25]:
# fig, ax = plt.subplots(figsize=(15,9))

# plot_roc_curve(fprs, tprs, aucs, "CNN")

### Confusion Matrix

In [26]:
# model_pred = np.argmax(model.predict(X_test), axis=1)

In [27]:
# fig, ax = plt.subplots(figsize=(15, 9))

# plot_confusion_matrix(le.inverse_transform(labels), y_test, model_pred, 'CNN')